In [ ]:
import numpy as np
import pandas as pd
import shap
import os, fnmatch
import matplotlib.pyplot as plt

import keras
from keras_preprocessing.image import ImageDataGenerator
from keras.models import load_model

In [ ]:
# Matplotlib in jupyter
%pylab inline
pylab.rcParams['figure.figsize'] = (12, 10)

### Load fine-tuned model
from `vgg_finetuning.py`

In [ ]:
# Load saved model incl. weights
model_final = load_model('2019-03-19_model_final.h5')

In [ ]:
# Take a look at the model architecture
model_final.summary()

### Load data

In [ ]:
# Load data
df_train = pd.read_pickle('/home/fabianmueller/pet_finder/data/prepared/train.pkl')

# Settings
BATCH_SIZE = 64

# Path and files
train_images_path = '/home/fabianmueller/pet_finder/data/raw/images/train_images'
train_images = fnmatch.filter(os.listdir(train_images_path), '*.jpg')

# Make binary target
target = 'adoptionspeed_bin'
df_train.loc[:, target] = np.where(df_train.loc[:, 'adoptionspeed'] <= 2, 'adopted', 'not_adopted')

# Create dict with id and target class
name_target_dict = df_train.set_index('petid')['adoptionspeed_bin'].to_dict()

# Empty dict for generator
dict_generator = {'filename': [], 'class': []}

# Go through images and save in generator dict
for name in train_images:
    short_name = name.split('-')[0]
    label = name_target_dict[short_name]

    dict_generator['filename'].append(name)
    dict_generator['class'].append(label)

# Convert dict to pd.DataFrame
df_generator = pd.DataFrame(dict_generator)

# Create generator with rescaling
data_gen = ImageDataGenerator(rescale=1. / 255)

# Training generator
train_generator = data_gen.flow_from_dataframe(dataframe=df_generator,
                                               directory=train_images_path,
                                               x_col='filename',
                                               y_col='class',
                                               target_size=(224, 224),
                                               class_mode='categorical',
                                               shuffle=True,
                                               batch_size=BATCH_SIZE)

### Prepare examples for model interpretation

In [ ]:
# Use this files as examples
examples = ['f394d4aad-5.jpg', 'd71310023-2.jpg', '1c92ce464-5.jpg']

In [ ]:
# Define background examples to take an expectation over
one_batch = train_generator.next()
background = one_batch[0]
background.shape

In [ ]:
# Define some helper functions

def get_example_batch(filename):
    '''
    Function for getting one file as batch from generator
    '''
    row = df_generator.loc[df_generator['filename'] == filename]
    
    gen = data_gen.flow_from_dataframe(dataframe=row,
                                       directory=train_images_path,
                                       x_col='filename',
                                       y_col='class',
                                       target_size=(224, 224),
                                       class_mode='categorical',
                                       shuffle=False,
                                       batch_size=1)
    
    batch = gen.next()
    return batch[0]

def map2layer(x, layer):
    '''
    Function for getting predictions from model for specific layer
    '''
    feed_dict = dict(zip([model_final.layers[0].input], [x.copy()]))
    return keras.backend.get_session().run(model_final.layers[layer].input, feed_dict)

def explain_and_plot(filename, layer):
    '''
    Function for getting explanations via shap values (for some layer)
    '''
    explain_me = get_example_batch(filename)
    
    e = shap.GradientExplainer(model=(model_final.layers[layer].input, model_final.layers[-1].output), 
                               data=map2layer(background.copy(), layer))
    
    shap_values, indexes = e.shap_values(map2layer(explain_me, layer), ranked_outputs=1)
    
    shap.image_plot(shap_values, explain_me)

In [ ]:
# Explain Kat
explain_and_plot(examples[0], layer=7)

In [ ]:
# Explain Danny
explain_and_plot(examples[1], layer=7)

In [ ]:
# Explain Beauty
explain_and_plot(examples[2], layer=7)